In [1]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/coref-model-2018.02.05.tar.gz")

/Users/frederikwarburg/anaconda3/envs/neuralcoref/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [8]:
import pandas as pd
import numpy as np
import spacy
from utils import spans, get_cluster_num, get_span_from_offset
from calculatescore import calculate_score, eval_gender, evaluate

In [9]:
data = pd.read_csv('data/gendered-pronoun-resolution/test_stage_1.tsv', sep = '\t')
data.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera


In [10]:
# NOTE THAT THIS TAKES A LONG TIME > 30 min

results = []
for index in range(len(data['Text'])):
    
    text = data['Text'][index]
    prediction = predictor.predict(
      document=text
    )

    pronoun_span = get_span_from_offset(text, data['Pronoun-offset'][index])
    A_span = get_span_from_offset(text, data['A-offset'][index])
    B_span = get_span_from_offset(text, data['B-offset'][index])

    pronoun_cluster_num = get_cluster_num(pronoun_span, prediction)
    A_cluster_num = get_cluster_num(A_span, prediction)
    B_cluster_num = get_cluster_num(B_span, prediction)

    A, B, N = 0, 0, 0
    for pronoun_pred in pronoun_cluster_num:
        if pronoun_pred in A_cluster_num: A = 1 
        if pronoun_pred in B_cluster_num: B = 1 

    if A == 0 and B == 0: N = 1
    
    results.append([A, B, N])
    
    if index % 10 == 0:
        print(index, " / ", len(data['Text']))


0  /  2000
10  /  2000
20  /  2000
30  /  2000
40  /  2000
50  /  2000
60  /  2000
70  /  2000
80  /  2000
90  /  2000
100  /  2000
110  /  2000
120  /  2000
130  /  2000
140  /  2000
150  /  2000
160  /  2000
170  /  2000
180  /  2000
190  /  2000
200  /  2000
210  /  2000
220  /  2000
230  /  2000
240  /  2000
250  /  2000
260  /  2000
270  /  2000
280  /  2000
290  /  2000
300  /  2000
310  /  2000
320  /  2000
330  /  2000
340  /  2000
350  /  2000
360  /  2000
370  /  2000
380  /  2000
390  /  2000
400  /  2000
410  /  2000
420  /  2000
430  /  2000
440  /  2000
450  /  2000
460  /  2000
470  /  2000
480  /  2000
490  /  2000
500  /  2000
510  /  2000
520  /  2000
530  /  2000
540  /  2000
550  /  2000
560  /  2000
570  /  2000
580  /  2000
590  /  2000
600  /  2000
610  /  2000
620  /  2000
630  /  2000
640  /  2000
650  /  2000
660  /  2000
670  /  2000
680  /  2000
690  /  2000
700  /  2000
710  /  2000
720  /  2000
730  /  2000
740  /  2000
750  /  2000
760  /  2000
770  /  20

In [15]:
# Random guess
predictions = pd.DataFrame(results, columns=['A','B','NEITHER'])
predictions.to_csv('data/predictions_allen_default1.csv')
predictions.head()

,A,B,NEITHER
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1


In [16]:
calculate_score('data/right_answers.csv','data/predictions_allen_default1.csv')

17.00977982080348

In [17]:
precision, recall, f1 = evaluate('data/right_answers.csv', 'data/predictions_allen_default1.csv')
print(precision, recall, f1)

[0.61586314 0.67114094 0.15904366] [0.45308924 0.43243243 0.76119403] [0.52208306 0.52596976 0.26311264]


In [18]:
# Gold path
path_gold = 'data/right_answers.csv'

# Path full data #
path_full = 'data/gendered-pronoun-resolution/test_stage_1.tsv'
df = pd.read_csv(path_full, sep = '\t', index_col = 0) 

# Create predictions #
predictions = pd.DataFrame(results, columns=['A','B','NEITHER'], index = list(df.index))
predictions.to_csv('data/predictions_allen_default2.csv')
path_pred = 'data/predictions_allen_default2.csv'

male_sc, female_sc = eval_gender(path_gold, path_pred, path_full)

Male precision: A = 0.6192, B = 0.6706, Neither = 0.1735
Male recall: A = 0.5159817351598174, B = 0.5010893246187363, Neither = 0.6990291262135923
Male F1: A = 0.5629, B = 0.5736, Neither = 0.278 

Female precision: A = 0.6115107913669064, B = 0.6719367588932806, Neither = 0.1480804387568556
Female recall: A = 0.38990825688073394, B = 0.3648068669527897, Neither = 0.826530612244898
Female F1: A = 0.4762, B = 0.4729, Neither = 0.2512
